# Implementing gradient descent to train a network on graduate school admissions data

## Data cleanup

In [54]:
import numpy as np
import pandas as pd

In [55]:
admissions = pd.read_csv('binary.csv')
admissions[10:]

,admit,gre,gpa,rank
10,0,800,4.00,4
11,0,440,3.22,1
12,1,760,4.00,1
13,0,700,3.08,2
14,1,700,4.00,1
...,...,...,...,...
395,0,620,4.00,2
396,0,560,3.04,3
397,0,460,2.63,2
398,0,700,3.65,2


### One-Hot Encoding

We need to use dummy variables to encode rank, splitting the data into four new columns encoded with ones or zeros.

In [56]:
# Make dummy variables for rank
data = pd.concat([admissions, 
                  pd.get_dummies(admissions['rank'], 
                                 prefix='rank')], 
                 axis=1)
data[10:]

,admit,gre,gpa,rank,rank_1,rank_2,rank_3,rank_4
10,0,800,4.00,4,0,0,0,1
11,0,440,3.22,1,1,0,0,0
12,1,760,4.00,1,1,0,0,0
13,0,700,3.08,2,0,1,0,0
14,1,700,4.00,1,1,0,0,0
...,...,...,...,...,...,...,...,...
395,0,620,4.00,2,0,1,0,0
396,0,560,3.04,3,0,0,1,0
397,0,460,2.63,2,0,1,0,0
398,0,700,3.65,2,0,1,0,0


In [57]:
data = data.drop('rank', axis=1)
data[10:]

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
10,0,800,4.00,0,0,0,1
11,0,440,3.22,1,0,0,0
12,1,760,4.00,1,0,0,0
13,0,700,3.08,0,1,0,0
14,1,700,4.00,1,0,0,0
...,...,...,...,...,...,...,...
395,0,620,4.00,0,1,0,0
396,0,560,3.04,0,0,1,0
397,0,460,2.63,0,1,0,0
398,0,700,3.65,0,1,0,0


### Scaling the data

In [58]:
# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std          # to get Z-score
data[:10]

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,-1.798011,0.578348,0,0,1,0
1,1,0.625884,0.736008,0,0,1,0
2,1,1.837832,1.603135,1,0,0,0
3,1,0.452749,-0.525269,0,0,0,1
4,0,-0.586063,-1.208461,0,0,0,1
5,1,1.491561,-1.024525,0,1,0,0
6,1,-0.239793,-1.077078,1,0,0,0
7,0,-1.624876,-0.814312,0,1,0,0
8,1,-0.412928,0.000263,0,0,1,0
9,0,0.972155,1.392922,0,1,0,0


### Spiliting the data into Training and Testing

In [59]:
# Split off random 10% of the data for testing
np.random.seed(42)

sample = np.random.choice(data.index, 
                          size=int(len(data)*0.9), 
                          replace=False)

data, test_data = data.ix[sample], data.drop(sample)     # train data, test data

In [60]:
print("Number of training samples is", len(data))       
print("Number of testing samples is", len(test_data))

Number of training samples is 360
Number of testing samples is 40


In [61]:
print("train_data: ", "\n", data[:10], "\n")
print("test_data: ", "\n", test_data[:10])

train_data:  
      admit       gre       gpa  rank_1  rank_2  rank_3  rank_4
209      0 -0.066657  0.289305       0       1       0       0
280      0  0.625884  1.445476       0       1       0       0
33       1  1.837832  1.603135       0       0       1       0
210      0  1.318426 -0.131120       0       0       0       1
93       0 -0.066657 -1.208461       0       1       0       0
84       1 -0.759199  0.552071       0       0       1       0
329      0 -0.759199 -1.208461       0       0       0       1
94       1  0.625884  0.131646       0       1       0       0
266      0 -0.239793 -0.393886       0       0       0       1
126      1  0.106478  0.394412       1       0       0       0 

test_data:  
      admit       gre       gpa  rank_1  rank_2  rank_3  rank_4
20       0 -0.759199 -0.577822       0       0       1       0
21       1  0.625884  0.630901       0       1       0       0
48       0 -1.278605 -2.390908       0       0       0       1
50       0  0.452749  1.

## Split into features and targets

In [62]:
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

In [63]:
print(features[:10])
print(targets[:10])

          gre       gpa  rank_1  rank_2  rank_3  rank_4
209 -0.066657  0.289305       0       1       0       0
280  0.625884  1.445476       0       1       0       0
33   1.837832  1.603135       0       0       1       0
210  1.318426 -0.131120       0       0       0       1
93  -0.066657 -1.208461       0       1       0       0
84  -0.759199  0.552071       0       0       1       0
329 -0.759199 -1.208461       0       0       0       1
94   0.625884  0.131646       0       1       0       0
266 -0.239793 -0.393886       0       0       0       1
126  0.106478  0.394412       1       0       0       0
209    0
280    0
33     1
210    0
93     0
84     1
329    0
94     1
266    0
126    1
Name: admit, dtype: int64


## Programming
- More efficient solution without sigmoid_prime

In [72]:
import numpy as np
# from data_prep import features, targets, features_test, targets_test


def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

# TODO: We haven't provided the sigmoid_prime function like we did in
#       the previous lesson to encourage you to come up with a more
#       efficient solution. If you need a hint, check out the comments
#       in solution.py from the previous lecture.

# Use to same seed to make debugging easier
np.random.seed(42)

n_records, n_features = features.shape
last_loss = None

# Initialize weights
weights = np.random.normal(scale=1 / n_features**.5, size=n_features)

# Neural Network hyperparameters
epochs = 100
learnrate = 0.5

for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x, y in zip(features.values, targets):
        # Loop through all records, x is the input, y is the target

        # Note: We haven't included the h variable from the previous
        #       lesson. You can add it if you want, or you can calculate
        #       the h together with the output

        # TODO: Calculate the output
        # Activation of the output unit
        #   Notice we multiply the inputs and the weights (np.dot(x, weights)) here 
        #   rather than storing h as a separate variable 
        output = sigmoid(np.dot(x, weights))

        # TODO: Calculate the error
        # The error, the target minus the network output
        # error = -y*np.log(output) - (1 - y) * np.log(1-output)
        error = y - output # much simpler

        # TODO: Calculate the error term
        # The error term
        #   Notice we calulate the gradient of the sigmoid f'(h) here 
        #   instead of defining a separate sigmoid_prime function.
        #   This just makes it faster because we
        #   can re-use the result of the sigmoid function stored in
        #   the output variable 
        # error_term = (y - output) * output * (1 - output)
        error_term = error * output * (1 - output) # much simpler
        
        # TODO: Calculate the change in weights for this sample
        #       and add it to the total weight change
        del_w += error_term * x

    # TODO: Update weights using the learning rate and the average change in weights
    weights += learnrate * del_w / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss


# Calculate accuracy on test data
tes_out = sigmoid(np.dot(features_test, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Train loss:  0.26276093849966364
Train loss:  0.20928619409324895
Train loss:  0.20084292908073417
Train loss:  0.1986215647552789
Train loss:  0.19779851396686018
Train loss:  0.19742577912189863
Train loss:  0.19723507746241065
Train loss:  0.19712945625092465
Train loss:  0.19706766341315077
Train loss:  0.19703005801777368
Prediction accuracy: 0.725


In [71]:
# Alternatively,

In [70]:
import numpy as np
# from data_prep import features, targets, features_test, targets_test


def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

# TODO: We haven't provided the sigmoid_prime function like we did in
#       the previous lesson to encourage you to come up with a more
#       efficient solution. If you need a hint, check out the comments
#       in solution.py from the previous lecture.

# Use to same seed to make debugging easier
np.random.seed(42)

n_records, n_features = features.shape
last_loss = None

# Initialize weights
weights = np.random.normal(scale=1 / n_features**.5, size=n_features)

# Neural Network hyperparameters
epochs = 1000
learnrate = 0.5

for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x, y in zip(features.values, targets):
        # Loop through all records, x is the input, y is the target

        # Note: We haven't included the h variable from the previous
        #       lesson. You can add it if you want, or you can calculate
        #       the h together with the output

        # TODO: Calculate the output
        output = sigmoid(np.dot(x, weights))

        # TODO: Calculate the error
        error = -y*np.log(output) - (1 - y) * np.log(1-output)

        # TODO: Calculate the error term
        error_term = (y - output) * output * (1 - output)

        # TODO: Calculate the change in weights for this sample
        #       and add it to the total weight change
        del_w += error_term * x

    # TODO: Update weights using the learning rate and the average change in weights
    weights += learnrate * del_w / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss


# Calculate accuracy on test data
tes_out = sigmoid(np.dot(features_test, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Train loss:  0.26276093849966364
Train loss:  0.20928619409324895
Train loss:  0.20084292908073417
Train loss:  0.1986215647552789
Train loss:  0.19779851396686018
Train loss:  0.19742577912189863
Train loss:  0.19723507746241065
Train loss:  0.19712945625092465
Train loss:  0.19706766341315077
Train loss:  0.19703005801777368
Prediction accuracy: 0.725
